In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# 1. 导入 SummaryWriter
from torch.utils.tensorboard import SummaryWriter

# --- 实例化 SummaryWriter ---
# 这会创建一个 'runs/iris_fca_gnn' 文件夹来存放日志
writer = SummaryWriter('../runs/iris_fca_gnn')

# --- 1. 数据加载与预处理 (与你的代码相同) ---

# a) 加载节点特征矩阵 X (来自FCA处理后的二元上下文)
features_path = '../data/iris.data.cleaned.csv'
x_numpy = np.loadtxt(features_path, delimiter=',')
x = torch.tensor(x_numpy, dtype=torch.float)

# b) 加载邻接矩阵 A+ (来自FCA概念格) 并转换为 edge_index
adj_matrix_path = '../data/iris_A_plus.csv'
a_plus_numpy = np.loadtxt(adj_matrix_path, delimiter=',')
a_plus = torch.tensor(a_plus_numpy, dtype=torch.float)
# ==========================================================
# ====================  【核心修改】 ====================
#              让邻接矩阵变得稀疏
# ==========================================================
print(f"原始邻接矩阵中的非零元素数: {torch.count_nonzero(a_plus)}")
threshold = 10
a_plus[a_plus <= threshold] = 0
a_plus.fill_diagonal_(0)
print(f"阈值化后邻接矩阵中的非零元素数: {torch.count_nonzero(a_plus)}")
# ==========================================================
edge_index, edge_attr = dense_to_sparse(a_plus)

# c) 从原始 iris.data 文件加载标签 y
labels_path = '../data/iris.data'
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
iris_df = pd.read_csv(labels_path, header=None, names=column_names)
species_labels = iris_df['species'].values
encoder = LabelEncoder()
y_numpy = encoder.fit_transform(species_labels)
y = torch.tensor(y_numpy, dtype=torch.long)

num_nodes = x.shape[0]
if num_nodes != len(y):
    print(f"警告: 特征矩阵的节点数 ({num_nodes}) 与标签数 ({len(y)}) 不匹配。")
    y = y[:num_nodes]
    print(f"已将标签截取为 {len(y)} 个以匹配节点数。")

# d) 创建 PyTorch Geometric 的 Data 对象
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# e) 划分训练集和测试集 (4:1)
num_train = int(num_nodes * 0.8)
num_test = num_nodes - num_train
indices = torch.randperm(num_nodes)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[indices[:num_train]] = True
test_mask[indices[num_train:]] = True
data.train_mask = train_mask
data.test_mask = test_mask

print("--- 数据加载完成 ---")
print(data)
print(f"节点总数: {data.num_nodes}")
print(f"训练节点数: {data.train_mask.sum().item()}")
print(f"测试节点数: {data.test_mask.sum().item()}")
print(f"类别数: {len(np.unique(y_numpy))}")


# --- 2. GNN 模型定义 (与你的代码相同) ---
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# --- 3. 【修改】训练与评估 ---
model = GCN(num_node_features=data.num_node_features, num_classes=len(np.unique(y_numpy)))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# 将 train 和 test 函数修改为接收 epoch 参数
def train(epoch):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    # 2. 在训练函数中记录 Loss
    writer.add_scalar('Loss/train', loss.item(), epoch)
    
    return loss.item()

def evaluate(epoch):
    """一个通用的评估函数，可以计算任何数据集划分的准确率"""
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        
        # 计算训练集准确率
        correct_train = pred[data.train_mask] == data.y[data.train_mask]
        train_acc = int(correct_train.sum()) / int(data.train_mask.sum())
        
        # 计算测试集准确率
        correct_test = pred[data.test_mask] == data.y[data.test_mask]
        test_acc = int(correct_test.sum()) / int(data.test_mask.sum())

        # 3. 在评估函数中记录训练和测试准确率
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/test', test_acc, epoch)
        
        return train_acc, test_acc

print("\n--- 开始训练 ---")
for epoch in range(1, 201):
    loss = train(epoch)
    
    # 在每个 epoch 后都进行评估，以便记录每一轮的精度
    train_acc, test_acc = evaluate(epoch)
    
    if epoch % 20 == 0:
        # 打印时，我们直接使用评估函数返回的结果
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

# 4. 训练结束后关闭 writer
writer.close()

final_test_acc = evaluate(200)[1] # 获取最后一个 epoch 的测试精度
print(f'--- 训练完成 ---')
print(f'最终测试集准确率: {final_test_acc:.4f}')
print("\nTensorBoard 日志已写入 'runs' 文件夹。")

原始邻接矩阵中的非零元素数: 22500
阈值化后邻接矩阵中的非零元素数: 653
--- 数据加载完成 ---
Data(x=[150, 126], edge_index=[2, 653], edge_attr=[653], y=[150], train_mask=[150], test_mask=[150])
节点总数: 150
训练节点数: 120
测试节点数: 30
类别数: 3

--- 开始训练 ---
Epoch: 020, Loss: 0.1859, Train Acc: 0.9917, Test Acc: 1.0000
Epoch: 040, Loss: 0.0620, Train Acc: 1.0000, Test Acc: 1.0000
Epoch: 060, Loss: 0.0321, Train Acc: 1.0000, Test Acc: 0.9667
Epoch: 080, Loss: 0.0181, Train Acc: 1.0000, Test Acc: 0.9667
Epoch: 100, Loss: 0.0156, Train Acc: 1.0000, Test Acc: 0.9667
Epoch: 120, Loss: 0.0161, Train Acc: 1.0000, Test Acc: 0.9667
Epoch: 140, Loss: 0.0441, Train Acc: 1.0000, Test Acc: 0.9667
Epoch: 160, Loss: 0.0087, Train Acc: 1.0000, Test Acc: 0.9667
Epoch: 180, Loss: 0.0187, Train Acc: 1.0000, Test Acc: 0.9667
Epoch: 200, Loss: 0.0220, Train Acc: 1.0000, Test Acc: 0.9667
--- 训练完成 ---
最终测试集准确率: 0.9667

TensorBoard 日志已写入 'runs' 文件夹。
